# Character Recognition Model

In [1]:
import tensorflow as tf

In [3]:
import pandas as pd

In [2]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [4]:
import tensorflowjs as tfjs

Load in Data

In [5]:
def df_processing(input_set):
    features = input_set.copy()
    labels = features.pop(features.columns[0])
    
    #one hot non 0 feature values to 1
    #and make them floats
    feature_col_names = list(features.columns.values)
    features[feature_col_names] = features.where(features[feature_col_names] == 0, 1).astype('float32')
    
    output_tensor = tf.data.Dataset.from_tensor_slices((features, labels))
    
    return output_tensor

In [6]:
train_set = pd.read_csv('../Data/emnist-letters-train.csv')

In [7]:
train_tensor = df_processing(train_set)

In [8]:
#check if the tensor worked
for feat, targ in train_tensor.take(1):
    print('Features: {}, Target: {}'.format(feat, targ))

Features: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [9]:
#shuffle and batch
train_tensor_shuffle_batch = train_tensor.shuffle(len(train_set)).batch(15)

Model

In [10]:
def get_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(100, activation = 'relu'),
        tf.keras.layers.Dense(50, activation = 'relu'),
        tf.keras.layers.Dense(27) #outputs [0, 27)
    ])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])
    
    return model

In [11]:
letter_model = get_model()

In [12]:
letter_model.fit(train_tensor_shuffle_batch, epochs = 10)

Epoch 1/10
5920/5920 [==============================] - 19s 3ms/step - loss: 0.8689 - sparse_categorical_accuracy: 0.7349
Epoch 2/10
5920/5920 [==============================] - 20s 3ms/step - loss: 0.5422 - sparse_categorical_accuracy: 0.8280
Epoch 3/10
5920/5920 [==============================] - 22s 4ms/step - loss: 0.4635 - sparse_categorical_accuracy: 0.8499
Epoch 4/10
5920/5920 [==============================] - 17s 3ms/step - loss: 0.4213 - sparse_categorical_accuracy: 0.8634
Epoch 5/10
5920/5920 [==============================] - 18s 3ms/step - loss: 0.3910 - sparse_categorical_accuracy: 0.8715
Epoch 6/10
5920/5920 [==============================] - 20s 3ms/step - loss: 0.3629 - sparse_categorical_accuracy: 0.8793
Epoch 7/10
5920/5920 [==============================] - 19s 3ms/step - loss: 0.3468 - sparse_categorical_accuracy: 0.8844
Epoch 8/10
5920/5920 [==============================] - 23s 3ms/step - loss: 0.3319 - sparse_categorical_accuracy: 0.8879
Epoch 9/10
5920/5920 [==

Save the Model

In [14]:
#convert keras to tf model and export it
tfjs.converters.save_keras_model(letter_model, "../model")